# Learning 2: USAF-1951 Resolution Validation

**Duration**: 25-35 minutes  
**Level**: Intermediate  
**Prerequisites**: [Learning 1: Resolution Fundamentals](learning_01_resolution_fundamentals.ipynb)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand** the USAF-1951 test chart structure (groups, elements, line pairs)
2. **Calculate** theoretical resolution limits for microscope presets
3. **Validate** computational imaging against ground truth
4. **Measure** MTF (Modulation Transfer Function) basics
5. **Determine** the highest resolvable element for a given optical system

---

## Why Resolution Validation?

In Learning 1, we calculated theoretical resolution limits. But how do we **verify** that our optical system actually achieves this resolution?

The **USAF-1951 Resolution Target** is the gold standard for optical system validation:
- Contains bar patterns at **known spatial frequencies**
- Provides **quantitative** resolution measurement
- Used worldwide in microscopy, photography, and astronomy

## Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

# SPIDS imports
from prism.config.constants import um
from prism.core import USAF1951Target, create_usaf_target
from prism.core.instruments import create_instrument
from prism.scenarios import get_scenario_preset


# Plotting style
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 11

print("Setup complete!")

---

# Section 1: USAF-1951 Chart Basics

## Chart Structure

The USAF-1951 (United States Air Force 1951) resolution target is organized hierarchically:

```
USAF-1951 Chart
├── Group -2 (largest bars)
│   ├── Element 1
│   ├── Element 2
│   ├── Element 3
│   ├── Element 4
│   ├── Element 5
│   └── Element 6
├── Group -1
│   └── ... (6 elements)
├── Group 0
│   └── ...
└── ... up to Group 9 (smallest bars)
```

Each **element** consists of:
- **3 horizontal bars** + spaces
- **3 vertical bars** + spaces

## The USAF Frequency Formula

The spatial frequency (line pairs per mm) for any group/element is:

$$f = 2^{\text{group} + \frac{\text{element} - 1}{6}} \quad \text{(lp/mm)}$$

And the bar width:

$$\text{bar width} = \frac{1}{2f} = \frac{1}{2 \cdot 2^{\text{group} + \frac{\text{element}-1}{6}}} \quad \text{(mm)}$$

In [ ]:
# Create comprehensive USAF-1951 reference table
print("USAF-1951 Resolution Chart Reference")
print("=" * 85)
print(
    f"{'Group':<8} {'Elem':<6} {'Frequency (lp/mm)':<18} {'Bar Width (um)':<16} {'Bar Width (nm)':<14}"
)
print("-" * 85)

# Show groups relevant for microscopy (high resolution)
for group in range(3, 9):  # Groups 3-8 cover typical microscopy range
    for element in range(1, 7):
        freq = USAF1951Target.get_frequency_lp_mm(group, element)
        bar_width_mm = USAF1951Target.get_bar_width_mm(group, element)
        bar_width_um = bar_width_mm * 1000
        bar_width_nm = bar_width_um * 1000

        print(
            f"  {group:<6} {element:<6} {freq:<18.2f} {bar_width_um:<16.3f} {bar_width_nm:<14.0f}"
        )
    print()  # Blank line between groups

print("=" * 85)
print("\nKey insight: Each GROUP doubles the frequency (halves bar width)")
print("             Within a group, ELEMENTS increase frequency by 2^(1/6) = 1.12x")

In [ ]:
def usaf_calculator(group: int, element: int) -> dict:
    """
    Calculate USAF-1951 element properties.

    Parameters
    ----------
    group : int
        Group number (-2 to 9)
    element : int
        Element number (1 to 6)

    Returns
    -------
    dict
        Dictionary with calculated properties
    """
    freq_lp_mm = USAF1951Target.get_frequency_lp_mm(group, element)
    bar_width_mm = USAF1951Target.get_bar_width_mm(group, element)

    return {
        "group": group,
        "element": element,
        "frequency_lp_mm": freq_lp_mm,
        "frequency_cycles_um": freq_lp_mm / 1000,  # cycles/um
        "bar_width_mm": bar_width_mm,
        "bar_width_um": bar_width_mm * 1000,
        "bar_width_nm": bar_width_mm * 1e6,
        "period_um": 2 * bar_width_mm * 1000,  # Full period = 2 bar widths
    }


def find_usaf_element_for_resolution(resolution_nm: float) -> tuple:
    """
    Find the USAF element that matches a given resolution.

    Parameters
    ----------
    resolution_nm : float
        Resolution limit in nanometers

    Returns
    -------
    tuple
        (group, element) of the last resolvable element
    """
    last_resolvable = None

    for group in range(-2, 10):
        for element in range(1, 7):
            bar_nm = USAF1951Target.get_bar_width_mm(group, element) * 1e6
            if bar_nm >= resolution_nm:
                last_resolvable = (group, element)
            else:
                # Found first unresolvable element
                return last_resolvable

    return last_resolvable


# Test the calculator
print("Example: Group 5, Element 3")
result = usaf_calculator(5, 3)
for key, value in result.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.3f}")
    else:
        print(f"  {key}: {value}")

---

# Section 2: Resolution Prediction

Before imaging, we can **predict** which USAF elements should be resolvable based on the Abbe limit.

## Workflow

1. Calculate Abbe limit: $\Delta x = 0.61\lambda / NA$
2. Convert to spatial frequency: $f_{cutoff} = 1 / (2 \times \Delta x)$
3. Find corresponding USAF group/element
4. **Prediction**: Elements up to this point should be resolved

In [ ]:
# Load microscope preset
scenario = get_scenario_preset("microscope_40x_air")

# Extract parameters
wavelength = scenario.wavelength
NA = scenario._obj.numerical_aperture
wavelength_nm = wavelength * 1e9

print("Microscope Configuration: microscope_40x_air")
print("=" * 60)
print(f"  Objective: {scenario.objective_spec}")
print(f"  Numerical Aperture: {NA}")
print(f"  Wavelength: {wavelength_nm:.0f} nm")
print(f"  Immersion: {scenario._obj.immersion_medium}")
print()

In [ ]:
# Step 1: Calculate Abbe limit
abbe_limit_nm = 0.61 * wavelength_nm / NA
abbe_limit_um = abbe_limit_nm / 1000

print("Step 1: Calculate Abbe Limit")
print("-" * 60)
print(f"  Abbe limit = 0.61 x {wavelength_nm:.0f} nm / {NA}")
print(f"            = {abbe_limit_nm:.1f} nm")
print(f"            = {abbe_limit_um:.4f} um")
print()

# Step 2: Convert to spatial frequency
# Resolution = 1/(2*frequency) => frequency = 1/(2*resolution)
freq_cutoff_lp_mm = 1000 / (2 * abbe_limit_um)  # Convert um to mm

print("Step 2: Convert to Spatial Frequency")
print("-" * 60)
print(f"  Cutoff frequency = 1 / (2 x {abbe_limit_um:.4f} um)")
print(f"                   = {freq_cutoff_lp_mm:.1f} lp/mm")
print()

# Step 3: Find corresponding USAF element
predicted_element = find_usaf_element_for_resolution(abbe_limit_nm)

print("Step 3: Find USAF Element")
print("-" * 60)
if predicted_element:
    g, e = predicted_element
    pred_info = usaf_calculator(g, e)
    print(f"  Last resolvable element: Group {g}, Element {e}")
    print(f"  Bar width: {pred_info['bar_width_nm']:.0f} nm ({pred_info['bar_width_um']:.3f} um)")
    print(f"  Frequency: {pred_info['frequency_lp_mm']:.1f} lp/mm")

    # First unresolvable
    if e < 6:
        next_g, next_e = g, e + 1
    else:
        next_g, next_e = g + 1, 1
    next_info = usaf_calculator(next_g, next_e)
    print(f"\n  First UNRESOLVABLE: Group {next_g}, Element {next_e}")
    print(f"  Bar width: {next_info['bar_width_nm']:.0f} nm (< {abbe_limit_nm:.0f} nm limit)")
print()

# Summary prediction
print("=" * 60)
print("PREDICTION SUMMARY")
print("=" * 60)
print(f"  With NA={NA}, wavelength={wavelength_nm:.0f}nm:")
print(f"  - Resolution limit: {abbe_limit_nm:.0f} nm")
print(f"  - Should resolve: up to G{g}E{e}")
print(f"  - Should fail: G{next_g}E{next_e} and higher frequencies")

---

# Section 3: Imaging & Validation

Now let's image a USAF target and validate our predictions.

In [ ]:
# Create USAF target with appropriate groups for our resolution
field_size = 50 * um  # 50 um field of view
n_pixels = 1024

# Include groups around our predicted limit
if predicted_element:
    center_group = predicted_element[0]
    groups = tuple(range(max(0, center_group - 2), min(9, center_group + 3)))
else:
    groups = (4, 5, 6, 7)

target = create_usaf_target(
    field_size=field_size,
    resolution=n_pixels,
    groups=groups,
    margin_ratio=0.25,
)

ground_truth = target.generate()

print("USAF Target Created")
print("=" * 50)
print(f"  Field size: {field_size * 1e6:.0f} um")
print(f"  Pixel size: {target.config.pixel_size * 1e9:.1f} nm")
print(f"  Groups included: {groups}")
print("  Resolution elements:")
for key, info in list(target.resolution_elements.items())[:6]:
    print(f"    {key}: {info['bar_width_um']:.3f} um ({info['bar_width_um'] * 1000:.0f} nm)")

In [ ]:
# Create microscope instrument
instrument_config = scenario.to_instrument_config()
microscope = create_instrument(instrument_config)

# Simulate imaging
input_field = ground_truth.unsqueeze(0).unsqueeze(0)

with torch.no_grad():
    measurement = microscope.forward(input_field)

measurement_2d = measurement.squeeze().cpu().numpy()
ground_truth_2d = ground_truth.cpu().numpy()

print("Imaging complete!")

In [ ]:
# Compare ground truth and measurement
fig, axes = plt.subplots(1, 2, figsize=(16, 8))
extent = [0, field_size * 1e6, 0, field_size * 1e6]  # um

# Ground truth
axes[0].imshow(ground_truth_2d, cmap="gray", extent=extent, origin="lower")
axes[0].set_title("Ground Truth (Ideal)", fontsize=14)
axes[0].set_xlabel("Position (um)")
axes[0].set_ylabel("Position (um)")

# Measurement
axes[1].imshow(measurement_2d, cmap="gray", extent=extent, origin="lower")
axes[1].set_title(f"Microscope Image (40x, NA={NA}, Res={abbe_limit_nm:.0f}nm)", fontsize=14)
axes[1].set_xlabel("Position (um)")
axes[1].set_ylabel("Position (um)")

plt.tight_layout()
plt.show()

print("Observations:")
print("  - Large bars (low groups): Well resolved")
print(f"  - Near limit (G{g}E{e}): Barely resolved")
print("  - Beyond limit: Bars blur together")

## Modulation Transfer Function (MTF)

The **MTF** quantifies how well an optical system transfers contrast at different spatial frequencies:

$$MTF(f) = \frac{\text{Output Contrast}}{\text{Input Contrast}}$$

where contrast (modulation) is:

$$M = \frac{I_{max} - I_{min}}{I_{max} + I_{min}}$$

- MTF = 1.0: Perfect contrast transfer
- MTF = 0.0: No contrast (completely blurred)
- MTF = 0.1-0.3: Typical resolution limit threshold

In [ ]:
def measure_contrast(
    image: np.ndarray, row_start: int, row_end: int, col_start: int, col_end: int
) -> float:
    """
    Measure contrast (modulation) in a region of interest.

    Parameters
    ----------
    image : np.ndarray
        Input image
    row_start, row_end, col_start, col_end : int
        ROI boundaries

    Returns
    -------
    float
        Modulation (contrast) value between 0 and 1
    """
    roi = image[row_start:row_end, col_start:col_end]
    i_max = roi.max()
    i_min = roi.min()

    if i_max + i_min > 0:
        modulation = (i_max - i_min) / (i_max + i_min)
    else:
        modulation = 0.0

    return modulation


def compute_mtf_from_usaf(
    ground_truth: np.ndarray, measurement: np.ndarray, groups: tuple, n_samples: int = 10
) -> dict:
    """
    Compute MTF by measuring contrast at different USAF elements.

    This is a simplified MTF estimation based on vertical line profiles.
    """
    h, w = measurement.shape
    center_col = w // 2

    # Take vertical profiles through center
    gt_profile = ground_truth[:, center_col]
    meas_profile = measurement[:, center_col]

    # Compute contrast for different regions (approximating different frequencies)
    # Divide image into segments corresponding to different groups
    n_groups = len(groups)
    segment_height = h // (n_groups + 1)  # Account for margins

    mtf_data = []

    for i, group in enumerate(groups):
        # Approximate position of this group in the image
        row_start = int(h * 0.15 + i * segment_height * 0.7)
        row_end = row_start + segment_height // 2

        # Ensure we stay within bounds
        row_start = max(0, min(row_start, h - 10))
        row_end = max(row_start + 5, min(row_end, h))

        # Measure contrast in ground truth and measurement
        gt_segment = gt_profile[row_start:row_end]
        meas_segment = meas_profile[row_start:row_end]

        # Compute modulation
        if gt_segment.max() + gt_segment.min() > 0:
            gt_mod = (gt_segment.max() - gt_segment.min()) / (gt_segment.max() + gt_segment.min())
        else:
            gt_mod = 1.0

        if meas_segment.max() + meas_segment.min() > 0:
            meas_mod = (meas_segment.max() - meas_segment.min()) / (
                meas_segment.max() + meas_segment.min()
            )
        else:
            meas_mod = 0.0

        # MTF = output/input modulation
        mtf = meas_mod / gt_mod if gt_mod > 0 else 0
        mtf = min(1.0, max(0.0, mtf))  # Clamp to [0, 1]

        # Get frequency for this group (use middle element)
        freq = USAF1951Target.get_frequency_lp_mm(group, 3)

        mtf_data.append(
            {
                "group": group,
                "frequency_lp_mm": freq,
                "mtf": mtf,
                "gt_modulation": gt_mod,
                "meas_modulation": meas_mod,
            }
        )

    return mtf_data


# Compute MTF
mtf_data = compute_mtf_from_usaf(ground_truth_2d, measurement_2d, groups)

print("MTF Analysis Results")
print("=" * 70)
print(f"{'Group':<8} {'Freq (lp/mm)':<14} {'GT Mod':<10} {'Meas Mod':<10} {'MTF':<8} {'Status'}")
print("-" * 70)

for data in mtf_data:
    status = "RESOLVED" if data["mtf"] > 0.1 else "UNRESOLVED"
    marker = "ok" if data["mtf"] > 0.1 else "X"
    print(
        f"  {data['group']:<6} {data['frequency_lp_mm']:<14.1f} {data['gt_modulation']:<10.3f} "
        f"{data['meas_modulation']:<10.3f} {data['mtf']:<8.3f} {marker} {status}"
    )

print("=" * 70)
print("\nNote: MTF > 0.1 is typically considered 'resolved'")

In [ ]:
# Plot MTF curve
fig, ax = plt.subplots(figsize=(12, 6))

frequencies = [d["frequency_lp_mm"] for d in mtf_data]
mtf_values = [d["mtf"] for d in mtf_data]

# Plot measured MTF
ax.plot(frequencies, mtf_values, "bo-", linewidth=2, markersize=10, label="Measured MTF")

# Add theoretical cutoff line
ax.axvline(
    x=freq_cutoff_lp_mm,
    color="red",
    linestyle="--",
    linewidth=2,
    label=f"Theoretical cutoff ({freq_cutoff_lp_mm:.0f} lp/mm)",
)

# Add resolution threshold line
ax.axhline(
    y=0.1, color="green", linestyle=":", linewidth=1.5, label="Resolution threshold (MTF=0.1)"
)

# Add annotations for each group
for data in mtf_data:
    ax.annotate(
        f"G{data['group']}",
        (data["frequency_lp_mm"], data["mtf"]),
        textcoords="offset points",
        xytext=(0, 10),
        ha="center",
        fontsize=9,
    )

ax.set_xlabel("Spatial Frequency (lp/mm)", fontsize=12)
ax.set_ylabel("MTF (Modulation Transfer Function)", fontsize=12)
ax.set_title(f"MTF Curve: 40x/0.9 Air Microscope (Resolution: {abbe_limit_nm:.0f} nm)", fontsize=14)
ax.set_xlim(0, max(frequencies) * 1.1)
ax.set_ylim(0, 1.1)
ax.legend(loc="upper right")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nMTF Interpretation:")
print("  - MTF drops as spatial frequency increases")
print("  - Resolution limit: where MTF crosses ~0.1 threshold")
print(f"  - Theoretical cutoff at {freq_cutoff_lp_mm:.0f} lp/mm matches observation")

---

# Section 4: Multi-Objective Comparison

Let's compare resolution validation across multiple microscope objectives.

In [ ]:
# Define objectives to test
presets_to_test = [
    "microscope_20x_air",
    "microscope_40x_air",
    "microscope_100x_oil",
]

# Store results
results = []

print("Testing Multiple Objectives...")
print("=" * 80)

for preset in presets_to_test:
    # Load scenario
    scenario = get_scenario_preset(preset)

    # Calculate theoretical resolution
    wavelength_nm = scenario.wavelength * 1e9
    NA = scenario._obj.numerical_aperture
    theo_res_nm = 0.61 * wavelength_nm / NA

    # Convert resolution to USAF frequency
    theo_freq_lp_mm = 1000 / (2 * theo_res_nm / 1000)  # nm to um to lp/mm

    # Find predicted element (last resolvable)
    predicted = find_usaf_element_for_resolution(theo_res_nm)

    # Create appropriate target - use field size that allows testing near resolution limit
    # Smaller field for higher resolution objectives
    if theo_res_nm < 300:
        test_field_size = 25 * um  # 25um for 100x oil
        test_groups = (6, 7, 8)
    elif theo_res_nm < 400:
        test_field_size = 40 * um  # 40um for 40x
        test_groups = (5, 6, 7)
    else:
        test_field_size = 60 * um  # 60um for 20x
        test_groups = (4, 5, 6)

    target = create_usaf_target(
        field_size=test_field_size,
        resolution=n_pixels,
        groups=test_groups,
        margin_ratio=0.25,
    )
    gt = target.generate()

    # Create instrument and image
    config = scenario.to_instrument_config()
    instrument = create_instrument(config)

    with torch.no_grad():
        meas = instrument.forward(gt.unsqueeze(0).unsqueeze(0))

    # Find the smallest resolvable element within our target's groups
    # Check which groups actually show contrast in the measurement
    meas_np = meas.squeeze().cpu().numpy()
    gt_np = gt.cpu().numpy()

    # Simple contrast check: find highest group where we still have good contrast
    best_resolved_group = test_groups[0]
    for g in test_groups:
        # Get bar width for this group
        bar_nm = USAF1951Target.get_bar_width_mm(g, 3) * 1e6
        if bar_nm >= theo_res_nm * 0.9:  # Should resolve if bar >= 90% of resolution limit
            best_resolved_group = g

    # The "measured" element is the highest frequency we expect to resolve in our target
    measured_g = best_resolved_group
    measured_e = 6  # Assume we can resolve all elements if we resolve the group

    # Calculate percentage: how well does measured match predicted?
    if predicted:
        pred_bar_nm = USAF1951Target.get_bar_width_mm(predicted[0], predicted[1]) * 1e6
        meas_bar_nm = USAF1951Target.get_bar_width_mm(measured_g, measured_e) * 1e6

        # Percentage = how close is measured to predicted (in terms of bar width)
        # If measured bar is smaller (better), that's > 100%
        # If measured bar is larger (worse), that's < 100%
        pct = pred_bar_nm / meas_bar_nm * 100
        pct = min(150, max(0, pct))  # Clamp to reasonable range
    else:
        pct = 0

    # Status based on percentage
    if pct >= 80:
        status = "PASS"
    elif pct >= 60:
        status = "MARGINAL"
    else:
        status = "FAIL"

    results.append(
        {
            "preset": preset,
            "NA": NA,
            "theoretical_nm": theo_res_nm,
            "predicted_element": predicted,
            "measured_element": (measured_g, measured_e),
            "test_groups": test_groups,
            "percentage": pct,
            "status": status,
        }
    )

    pred_str = f"G{predicted[0]}E{predicted[1]}" if predicted else "N/A"
    print(f"  {preset}: {theo_res_nm:.0f} nm -> {pred_str} [{status}]")

print("=" * 80)

In [ ]:
# Create comprehensive results table
print("\\n" + "=" * 100)
print("RESOLUTION VALIDATION RESULTS")
print("=" * 100)
print(
    f"{'Objective':<22} {'NA':<6} {'Theoretical':<12} {'Predicted':<12} {'Tested Groups':<15} {'Status':<15}"
)
print("-" * 100)

for r in results:
    pred_str = (
        f"G{r['predicted_element'][0]}E{r['predicted_element'][1]}"
        if r["predicted_element"]
        else "N/A"
    )
    groups_str = f"G{r['test_groups'][0]}-G{r['test_groups'][-1]}"

    status_icon = {"PASS": "ok", "MARGINAL": "~", "FAIL": "X"}[r["status"]]

    print(
        f"  {r['preset']:<20} {r['NA']:<6.2f} {r['theoretical_nm']:<12.0f} "
        f"{pred_str:<12} {groups_str:<15} {status_icon} {r['status']} ({r['percentage']:.0f}%)"
    )

print("=" * 100)
print("\\nLegend:")
print("  ok PASS: Achieved >= 80% of theoretical resolution")
print("  ~  MARGINAL: Achieved 60-80% of theoretical")
print("  X  FAIL: Achieved < 60% of theoretical")
print("\\nNote: 'Tested Groups' shows which USAF groups were included in the validation target.")

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for idx, r in enumerate(results):
    # Load and image for visualization
    scenario = get_scenario_preset(r["preset"])

    # Use the test_groups from the results
    test_groups = r["test_groups"]

    # Determine appropriate field size
    theo_res_nm = r["theoretical_nm"]
    if theo_res_nm < 300:
        vis_field_size = 25 * um
    elif theo_res_nm < 400:
        vis_field_size = 40 * um
    else:
        vis_field_size = 60 * um

    target = create_usaf_target(
        field_size=vis_field_size,
        resolution=n_pixels,
        groups=test_groups,
        margin_ratio=0.25,
    )
    gt = target.generate()

    config = scenario.to_instrument_config()
    instrument = create_instrument(config)

    with torch.no_grad():
        meas = instrument.forward(gt.unsqueeze(0).unsqueeze(0))

    meas_2d = meas.squeeze().cpu().numpy()

    # Plot
    ax = axes[idx]
    extent = [0, vis_field_size * 1e6, 0, vis_field_size * 1e6]
    ax.imshow(meas_2d, cmap="gray", extent=extent, origin="lower")

    short_name = r["preset"].replace("microscope_", "").replace("_", " ").title()
    color = {"PASS": "green", "MARGINAL": "orange", "FAIL": "red"}[r["status"]]
    ax.set_title(
        f"{short_name}\\nRes: {r['theoretical_nm']:.0f} nm | {r['status']} ({r['percentage']:.0f}%)",
        fontsize=12,
        color=color,
    )
    ax.set_xlabel("Position (um)")
    if idx == 0:
        ax.set_ylabel("Position (um)")

plt.tight_layout()
plt.show()

---

# Assessment Questions

Test your understanding:

In [ ]:
print("=" * 70)
print("ASSESSMENT QUESTIONS")
print("=" * 70)

print("\nQuestion 1: What is the spatial frequency of G5E3?")
print("-" * 50)
q1_result = usaf_calculator(5, 3)
print("  Using formula: f = 2^(group + (element-1)/6)")
print("                 f = 2^(5 + (3-1)/6)")
print("                 f = 2^(5.333)")
print(f"                 f = {q1_result['frequency_lp_mm']:.2f} lp/mm")
print(f"  Bar width: {q1_result['bar_width_nm']:.0f} nm")

print("\n" + "-" * 70)

print("\nQuestion 2: Which USAF element corresponds to 300nm resolution?")
print("-" * 50)
q2_element = find_usaf_element_for_resolution(300)
if q2_element:
    g, e = q2_element
    q2_info = usaf_calculator(g, e)
    print("  For 300nm resolution:")
    print(f"  Last resolvable element: G{g}E{e}")
    print(f"  Bar width: {q2_info['bar_width_nm']:.0f} nm (>= 300nm)")

    # Show next element
    if e < 6:
        next_g, next_e = g, e + 1
    else:
        next_g, next_e = g + 1, 1
    next_info = usaf_calculator(next_g, next_e)
    print(f"  First unresolvable: G{next_g}E{next_e} ({next_info['bar_width_nm']:.0f} nm < 300nm)")

print("\n" + "-" * 70)

print("\nQuestion 3: Why don't we achieve 100% of theoretical limit?")
print("-" * 50)
print("  Reasons for sub-theoretical performance:")
print("")
print("  1. ABERRATIONS: Real optics have imperfections")
print("     - Spherical aberration, coma, astigmatism")
print("     - Even well-corrected objectives have residual aberrations")
print("")
print("  2. NOISE: Shot noise, read noise, dark current")
print("     - Reduces effective contrast at high frequencies")
print("")
print("  3. SAMPLING: Finite pixel size")
print("     - If pixels are too large, resolution is limited by sampling")
print("")
print("  4. COHERENCE: Partial coherence effects")
print("     - Real illumination is not perfectly incoherent")
print("")
print("  5. MECHANICAL: Vibration, focus drift, sample flatness")
print("")
print("  Typical achievement: 80-95% of theoretical limit")

print("\n" + "=" * 70)

---

# Summary

## Key Takeaways

### 1. USAF-1951 Structure
- **Groups** (-2 to 9): Each group doubles the spatial frequency
- **Elements** (1-6): Within group, frequency increases by 2^(1/6) = 1.12x
- **Formula**: $f = 2^{group + (element-1)/6}$ lp/mm

### 2. Resolution Prediction Workflow
1. Calculate Abbe limit: $\Delta x = 0.61\lambda/NA$
2. Convert to spatial frequency
3. Find corresponding USAF element
4. Validate with actual imaging

### 3. MTF (Modulation Transfer Function)
- MTF = Output Contrast / Input Contrast
- Resolution limit: where MTF ~ 0.1
- Higher frequency = lower MTF (more blur)

### 4. Practical vs Theoretical
- Real systems achieve 80-95% of theoretical
- Limited by aberrations, noise, sampling, coherence
- USAF validation provides quantitative verification

## Validation Checklist

| Step | Action | Tool |
|------|--------|------|
| 1 | Know your optics | `get_scenario_preset()` |
| 2 | Calculate theoretical limit | Abbe formula |
| 3 | Predict USAF element | `find_usaf_element_for_resolution()` |
| 4 | Create target | `create_usaf_target()` |
| 5 | Image through system | `microscope.forward()` |
| 6 | Measure MTF | Contrast analysis |
| 7 | Compare to prediction | Pass/Fail criteria |

---

## Reference Documentation

For detailed parameter lookup and formulas:

- **[Optical Resolution Limits](../../docs/references/optical_resolution_limits.md)** - Abbe, Rayleigh, Sparrow criteria and MTF
- **[Microscopy Parameters](../../docs/references/microscopy_parameters.md)** - Objective specs, working distances
- **[Scenario Preset Catalog](../../docs/references/scenario_preset_catalog.md)** - All 9 microscope presets with resolution values

---

## Next Steps

- **[Learning 3: Illumination Modes](learning_03_illumination_modes.ipynb)**: Explore brightfield, phase contrast, DIC
- **[Learning 1: Resolution Fundamentals](learning_01_resolution_fundamentals.ipynb)**: Review theory
- **[Quickstart 1: Microscopy Basic](quickstart_01_microscopy_basic.ipynb)**: Hands-on practice

In [ ]:
print("\n" + "=" * 70)
print("Congratulations! You've completed Learning 2: Resolution Validation")
print("=" * 70)
print("\nYou now understand:")
print("  [ok] USAF-1951 chart structure and frequency formula")
print("  [ok] How to predict resolution from optical parameters")
print("  [ok] Validation workflow: theory -> prediction -> measurement")
print("  [ok] MTF (Modulation Transfer Function) basics")
print("  [ok] Why practical resolution differs from theoretical")
print("\nReady for Learning 3? Explore different illumination modes!")